In [1]:
import numpy as np
import scipy
from numpy.linalg import norm

In [2]:
def hinge_loss(x,y):
    return max(0, 1- x*y)

In [23]:
import cvxopt
class SVM:
    
    def __init__(self, C = 1., loss_function = hinge_loss, kernel_function = None):
        self.C = C
        self.loss_function = hinge_loss
        self.kernel_function = lambda a,b : np.inner(a,b)#np.sum(np.minimum(a,b))
        
    
    def fit(self, X, y):
        
        n_samples = X.shape[0]
        n_features = X.shape[1]
        self.classes = np.unique(y)
        
        self.y_train = np.array([-1 if label == self.classes[0] else 1 for label in y])
        self.X_train = X
        y= self.y_train
        if (X.shape[0] != y.shape[0] or 1):
            print "X and y not of same size :",X.shape[0], y.shape[0]
        
        #compute the kernel matrix
        K = np.array([np.array([self.kernel_function(x,x2) for x2 in X])for x in X])
        self.K = K
        # Solves
        # min 1/2 x^T P x + q^T x
        # s.t.
        #  Gx \coneleq h
        #  Ax = b
        P = cvxopt.matrix(np.outer(y, y) * K)
        q = cvxopt.matrix(-1 * np.ones(n_samples))

        # -a_i \leq 0
        # TODO(tulloch) - modify G, h so that we have a soft-margin classifier
        G_std = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
        h_std = cvxopt.matrix(np.zeros(n_samples))

        # a_i \leq c
        G_slack = cvxopt.matrix(np.diag(np.ones(n_samples)))
        h_slack = cvxopt.matrix(np.ones(n_samples) * self.C)

        G = cvxopt.matrix(np.vstack((G_std, G_slack)))
        h = cvxopt.matrix(np.vstack((h_std, h_slack)))
        
        
        A = cvxopt.matrix(y, (1, n_samples))
        b = cvxopt.matrix(0.0)

        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        self.alpha = solution['x']
        self.b = np.mean([y[j] - np.sum([self.alpha[i]*y[i] * K[i,j] for i in range(n_samples)])
                         for j in range(n_samples)]).mean()
        return self
        
            
    def predict(self, X):
        prediction = []
        for i,x in enumerate(X):
            result = self.b
            for j in range(n_samples):
                result+= self.alpha[j]*self.y_train[j]*selph.kernel_function(selph.X_train[j],x)
                if(np.sign(result) <0):
                    prediction.append(np.sign(self.classes[0]))
                else:
                    prediction.append(np.sign(self.classes[1]))
                
        return prediction

In [24]:
#from svm import SVM
class multiclass_svm(object):
    def __init__(self, kernel, c):
        self.kernel = kernel
        self.c = c
    
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.trained_classifiers = [
            [SVM(C = self.c).fit(X[(y==label) | (y== label2)], y[(y==label)|(y== label2)])
                                                if label2>label else 0
                                                for label2 in self.classes] 
                                                for label in self.classes ]
        return self
        
    def predict(self, X):
        win_count = np.zeros((X.shape[0],len(self.classes)))
        for i, label1 in enumerate(self.classes):
            for j, label2 in enumerate(self.classes):
                if label2>label1:
                    y_test = self.trained_classifiers[i][j].predict(X)
                    for k, winner in enumerate(y_test):
                        if winner == label1:
                            win_count[k,i]+=1
                        else:
                            win_count[k,j]+=1
                            
        return [self.classes[np.argmax(row)] for row in win_count]

In [4]:

import pandas as pd
path=""
X_train =pd.read_csv(path+"Xtr.csv", header=None)
Y =pd.read_csv(path+"Ytr.csv")
X_test =pd.read_csv(path+"Xte.csv", header=None)

y_train = Y["Prediction"].values
X_train = X_train.values
X_test = X_test.values

from sklearn.decomposition import KernelPCA
kpca = KernelPCA(kernel = 'rbf', degree = 3, n_components= 35, gamma= 0.01)

X_train_kpca = kpca.fit_transform(X_train)
X_test_kpca = kpca.transform(X_test)

In [25]:
def min_kernel(a, b):
    return np.sum(np.minimum(a,b))

In [26]:
svm = multiclass_svm(min_kernel,10.)

In [27]:
#svm = SVC(C = 10.)
svm.fit(X_train,y_train)


X and y not of same size : 1021 1021


AttributeError: 'cvxopt.base.matrix' object has no attribute 'shape'

In [ ]:
X_train

In [19]:
from sklearn.metrics import accuracy_score
predicted_label = svm.predict(X_test)

print("SVM - Score on train_data : ", accuracy_score(y_train, svm.predict(X_train)))

('SVM - Score on train_data : ', 0.94159999999999999)


In [39]:
predicted_label

[7,
 6,
 1,
 0,
 4,
 1,
 4,
 9,
 6,
 9,
 0,
 6,
 9,
 0,
 1,
 3,
 4,
 7,
 3,
 4,
 9,
 6,
 6,
 3,
 9,
 0,
 7,
 4,
 0,
 1,
 3,
 1,
 3,
 6,
 7,
 2,
 7,
 1,
 2,
 1,
 1,
 7,
 4,
 4,
 3,
 3,
 1,
 2,
 9,
 4,
 6,
 3,
 5,
 3,
 6,
 0,
 4,
 1,
 9,
 1,
 7,
 8,
 9,
 1,
 7,
 9,
 6,
 4,
 3,
 0,
 7,
 0,
 2,
 8,
 1,
 7,
 3,
 7,
 1,
 7,
 9,
 6,
 2,
 7,
 8,
 4,
 7,
 3,
 6,
 1,
 3,
 6,
 1,
 3,
 1,
 4,
 1,
 8,
 6,
 9,
 6,
 0,
 5,
 4,
 9,
 9,
 2,
 1,
 9,
 6,
 8,
 1,
 3,
 9,
 1,
 9,
 9,
 4,
 9,
 2,
 3,
 4,
 7,
 6,
 7,
 9,
 0,
 5,
 8,
 9,
 6,
 6,
 5,
 2,
 8,
 1,
 0,
 1,
 6,
 4,
 6,
 7,
 3,
 1,
 9,
 1,
 8,
 2,
 0,
 1,
 4,
 3,
 5,
 5,
 1,
 5,
 4,
 6,
 3,
 1,
 4,
 6,
 5,
 4,
 4,
 3,
 4,
 1,
 1,
 9,
 9,
 8,
 3,
 3,
 2,
 1,
 1,
 8,
 1,
 8,
 1,
 8,
 5,
 0,
 3,
 9,
 8,
 3,
 0,
 1,
 1,
 1,
 0,
 9,
 0,
 1,
 1,
 6,
 9,
 2,
 3,
 6,
 1,
 1,
 1,
 3,
 9,
 3,
 2,
 9,
 4,
 5,
 3,
 1,
 9,
 3,
 3,
 1,
 7,
 3,
 7,
 2,
 2,
 7,
 1,
 2,
 8,
 1,
 1,
 7,
 3,
 1,
 8,
 9,
 1,
 9,
 2,
 1,
 4,
 1,
 3,
 3,
 8,
 9,
 2,
 3,
 0,
 6,
 4,
 1,
